In [2]:
import pandas as pd

In [3]:
import psycopg2
import getpass
# Lf8CvZDecmGdOrslR3a8

In [5]:
# enter the password (from keepass db) when prompted
conn = psycopg2.connect(host='map-fs1.ndph.ox.ac.uk',
                       port=5433,
                       dbname='dhs_data_pit',
                       user='dhs_reader',
                       password=getpass.getpass())

········


In [21]:
allFSTQuery = """SELECT surveyid, recordname as "RecordName", name as "Name", label, '' as "NewPublicTreat", '' as "NewAnyTreat" 
    FROM dhs_survey_specs.dhs_table_specs_flat 
    WHERE name like 'H32%' and label not like 'NA%' and label not like '%NA'"""
allAvailFSTQuery = f"""WITH questions AS 
    ({allFSTQuery}) 
    SELECT distinct(surveyid)::INTEGER FROM questions order by surveyid"""

In [7]:
allAvail = pd.read_sql_query(allAvailFSTQuery, conn)
allAvail

,surveyid
0,1
1,2
2,3
3,4
4,5
...,...
305,534
306,539
307,540
308,542


In [8]:
lastInputFile = r'G:\Shared drives\dhs-outputs\Standard_MAP_DHS_Outputs\Fever_Seeking_Treatment\GBD_Format\Latest_Run\value_mapping\Fever_Question_Mapping_20190528_all_KB.csv'

In [9]:
lastTimeQuestions = pd.read_csv(lastInputFile)

In [10]:
lastTimeSurveys = lastTimeQuestions.surveyid.unique()

In [11]:
surveysNotDone = allAvail[~allAvail.surveyid.isin(lastTimeSurveys)]
surveysNotDone

,surveyid
180,304
265,467
296,517
301,526
302,527
303,528
306,539
308,542
309,549


In [12]:
allSurveys = pd.read_sql_query('SELECT * FROM dhs_survey_specs.dhs_survey_listing', conn)
allSurveys.surveynum = allSurveys.surveynum.astype(int)
surveysNotDone.merge(allSurveys,left_on='surveyid', right_on='surveynum')

,surveyid_x,surveyid_y,dhs_countrycode,surveyyear,surveytype,surveystatus,countryname,surveyyearlabel,surveynum,indicatordata,...,numberofhouseholds,universeofwomen,numberofwomen,minagewomen,maxagewomen,universeofmen,numberofmen,minagemen,maxagemen,numberoffacilities
0,304,TZ2007AIS,TZ,2007,AIS,Completed,Tanzania,2007-08,304,1,...,8497,All Women,9343,15,49,All Men,6975,15,49,
1,467,MZ2015AIS,MZ,2015,AIS,Completed,Mozambique,2015,467,1,...,7169,All Women,7749,15,59,All Men,5283,15,59,
2,517,ML2018DHS,ML,2018,DHS,Completed,Mali,2018,517,1,...,9510,All Women,10519,15,49,All Men,4618,15,59,
3,526,BF2017MIS,BF,2017,MIS,Completed,Burkina Faso,2017-18,526,1,...,6322,All Women,7628,15,49,,,,,
4,527,MZ2018MIS,MZ,2018,MIS,Completed,Mozambique,2018,527,1,...,6196,All Women,6184,15,49,,,,,
5,528,NG2018DHS,NG,2018,DHS,Completed,Nigeria,2018,528,1,...,40427,All Women,41821,15,49,All Men,13311,15,59,
6,539,GN2018DHS,GN,2018,DHS,Completed,Guinea,2018,539,1,...,8200,All Women,8000,15,49,All Men,5000,15,59,
7,542,ZM2018DHS,ZM,2018,DHS,Completed,Zambia,2018,542,1,...,12831,All Women,13683,15,49,All Men,12132,15,59,
8,549,UG2018MIS,UG,2018,MIS,Completed,Uganda,2018-19,549,1,...,8351,All Women,8231,15,59,,,,,


In [13]:
newSurveyStr = surveysNotDone.surveyid.astype(str).str.cat(sep=',')

In [22]:
newFSTQuery = f"""{allFSTQuery} AND surveyid::INTEGER IN ({newSurveyStr})""" 
print(newFSTQuery)

SELECT surveyid, recordname as "RecordName", name as "Name", label, '' as "NewPublicTreat", '' as "NewAnyTreat" 
    FROM dhs_survey_specs.dhs_table_specs_flat 
    WHERE name like 'H32%' and label not like 'NA%' and label not like '%NA' AND surveyid::INTEGER IN (304,467,517,526,527,528,539,542,549)


In [23]:
newQuestions = pd.read_sql_query(newFSTQuery, conn)

In [24]:
newQuestions

,surveyid,RecordName,Name,label,NewPublicTreat,NewAnyTreat
0,467,REC43,H32A,Fever/cough: government hospital,,
1,467,REC43,H32B,Fever/cough: Mobile brigade,,
2,467,REC43,H32C,Fever/cough: APE (Agente Polivalente Elementar),,
3,467,REC43,H32D,Fever/cough: Other public,,
4,467,REC43,H32J,Fever/cough: private hospital/ clinic,,
...,...,...,...,...,...,...
157,542,REC4A,H32U,Fever/cough: market,,
158,542,REC4A,H32V,Fever/cough: itinerant drug seller,,
159,542,REC4A,H32X,Fever/cough: other,,
160,542,REC4A,H32Y,Fever/cough: no treatment,,


In [27]:
newQuestions.to_csv(r'G:\Shared drives\dhs-outputs\Standard_MAP_DHS_Outputs\Fever_Seeking_Treatment\GBD_Format\2020_05_26\value_mapping\new_surveys_fst_questions_20200526.csv', index=False)